# Code:

1. Install Anaconda which provides Jupyter. It is needed for Ipywidget

2. Install with Pip some libraries: pip install psycopg2 pip install ipywidgets jupyter nbextension enable --py widgetsnbextension
3. Run all the code (Cell/run all) . Cell 78: Interactive Plot

## Imports:

In [1]:
import json
import csv
import sys
import numpy as np
from datetime import *
import matplotlib.pyplot as plt
from matplotlib.pyplot import show
import matplotlib.image as mpimg
import dateutil.parser
from ipywidgets import interactive, HBox, Play, Text, jslink, VBox, SelectionSlider
import psycopg2
import ast

In [2]:
# %matplotlib notebook
%matplotlib inline

## Functions

In [3]:
def Timestudy(cursor, start, end):
    """This function helps to query the database to get the lines that 
    we want and to return the cursor will all data"""
    cursor.execute(f"select * from v1.observation where stream_id = 20220 AND data ->> 'phenomenonTime' > '{start}'" 
                   f"and data ->> 'phenomenonTime' < '{end}' LIMIT 10000")
    return cursor

In [4]:
def extract_array_json(cursor):
    """This function reads the cursor, extract the json files, and transfer everything 
    in a dictionnary with the timestamp as key and the heat map as value. 
    Time_HM is a list of all keys sorted. We also export the dictionnary and list with 
    datastamp under a string format for easier display later"""
    Dico_Time = {}
    Time_HM = []
    for row in cursor:
        json_data = str(row[1])
        b = ast.literal_eval(json_data)
        # On cherche toutes les clés possibles
        key_global = b.keys()
        key_result = b["result"].keys()

        #On regarde data
        HM = b['result']["density_map"]
        HM = np.array(HM)
        HM = HM.astype(float)

        #On met en forme le temps
        Time = dateutil.parser.parse(b["phenomenonTime"])
        Time = Time + timedelta(hours=2)
        
        if np.sum(HM)!= 0 :
            Time_HM.append(Time)
            Dico_Time[Time]= HM
            Time_HM=sorted(Time_HM)
    return Dico_Time, Time_HM

In [5]:
def Extract_HM_Hr (hr, Dico_Time ):
    """This function will get the heat map as a numpy array using the hr as key"""
    HM = np.array(Dico_Time[hr])
    return HM

In [6]:
def DrawMapLegend(HM, image) : 
    """
    This function draws a plot with the Heat Map on a blueprint and change the symbology
    The Extent of the background must be chosen to fit the sound heat map extent
    """
    #subplot
    fig, ax = plt.subplots(figsize=(20,30))
    # Ajout du blueprint
    map_img = mpimg.imread(image) 
    plt.imshow(map_img, zorder=0, extent=[0,30, 0, 20])

    # Interpolated Grid
    ax = plt.gca()
    ax.set_aspect('equal')
    h = ax.imshow(HM, cmap="rainbow", alpha=0.7,interpolation="bilinear", origin='lower', vmin=2, vmax=50)#
    for i in range(20):
        for j in range(30):
            text = ax.text(j, i, '{0:g}'.format(HM[i, j]),
                           ha="center", va="center", color="w", fontsize=14)

In [7]:
def VideoMapHr_keystr(hr):
    """
    Its Run the two functions : Extract_HM_Hr and DrawMapLegend
    It gives also the parameter of the first function(Dico_Time_str) to make it interactive using ipywidget
    """
    HM = Extract_HM_Hr(hr, Dico_Time_str)
    DrawMapLegend( HM = HM, image = 'Background3.png')

## Connect to Database

In [8]:
connection = psycopg2.connect(user = "postgres",
                                  password = "postgres",
                                  host = "192.168.0.1",
                                  port = "5432",
                                  database = "gost")
cursor = connection.cursor()

## Main:

In [9]:
start = datetime.now()
Timestudy(cursor, '2019-08-30T17:00:00.000Z', '2019-08-31T03:00:00.000Z')
stop = datetime.now()
duration = stop - start
print(duration)

0:00:00.003230


In [10]:
Dico_Time, Time_HM= extract_array_json(cursor)

In [11]:
def SetHeure(z):
    BandSlider.value=str(Time_HM_str[z])
    
# We change the datetime in string
lg = len(Time_HM)  
Time_HM_str = [str(obj)[:-13] for obj in Time_HM ]
Dico_Time_str={}
for key in Dico_Time.keys():
    Dico_Time_str[str(key)[:-13]] = Dico_Time[key]


BandSlider = SelectionSlider(options=Time_HM_str,continuous_update=False
                             , description= "Time FR", layout ={'width': '600px'} )


play = Play(min=0,max=lg, interval=300)
SoundHeatMap = interactive(VideoMapHr_keystr,hr=BandSlider)
interactive(SetHeure, z = play)
test = VBox([ play,SoundHeatMap])
test 